# Building a simple autoencoder

Based on the ability to generate UK car registration numbers, one can build a dataset of training and test data. With a suitable vectorizer, one can go ahead and make an autoencoder.

### References

https://blog.keras.io/building-autoencoders-in-keras.html

In [1]:
import sys
import numpy as np
import keras
from keras import regularizers
from keras import layers
import random

if '..' not in sys.path:
    sys.path.append('..')
from car_reg_generator.car_reg_generator.uk_reg import UkRegGenerator
from car_reg_generator.car_reg_generator.uk_reg import UkRegBowVectorizer

### Data generation

In [2]:
n_train = 10000
n_test = 1000

random.seed(0)
g = UkRegGenerator()
v = UkRegBowVectorizer()

train_strs = [g.get_reg() for _ in range(n_train)]
train_vecs = np.array([v.vectorize(x) for x in train_strs])
test_strs = [g.get_reg() for _ in range(n_test)]
test_vecs = np.array([v.vectorize(x) for x in test_strs])

vec_length = len(train_vecs[0])
print(vec_length)

252


### Standard autoencoder

Design the network. A single dense layer for the encoder and decoder needs an encoding dimension of around 100. Having more layers allows a smaller encoding dimension, although convergence during learning starts to become unstable.

In [21]:
encoding_dim = 30
input_reg = keras.Input(shape=(vec_length,))
# encoded = layers.Dense(encoding_dim, activation='relu', activity_regularizer=regularizers.l1(10e-5))(input_reg)
encoded = layers.Dense(vec_length, activation='relu')(input_reg)
encoded = layers.Dense(encoding_dim, activation='relu')(encoded)
encoded = layers.Dense(encoding_dim, activation='relu')(encoded)
encoder = keras.Model(input_reg, encoded)

encoded_input = keras.Input(shape=(encoding_dim,))
decoded = layers.Dense(encoding_dim, activation='relu')(encoded_input)
decoded = layers.Dense(vec_length, activation='relu')(decoded)
decoded = layers.Dense(vec_length, activation='sigmoid')(decoded)
decoder = keras.Model(encoded_input, decoded)

autoencoder = keras.Model(input_reg, decoder(encoder(input_reg)))

Do some learning. For shallow networks, almost any optimizer and loss work. For deeper networks, poor choices almost certainly yield failed convergence on the loss function. Even good choices yield convergence on the loss function only on some runs.

In [24]:
autoencoder.compile(optimizer='adam', loss='categorical_crossentropy')
autoencoder.fit(train_vecs, train_vecs,
                epochs=100,
                batch_size=256,
                shuffle=True,
                validation_data=(test_vecs, test_vecs))

Epoch 1/100
40/40 [==============================] - 1s 25ms/step - loss: 15.8011 - val_loss: 14.8478
Epoch 2/100
40/40 [==============================] - 1s 20ms/step - loss: 14.5325 - val_loss: 14.6482
Epoch 3/100
40/40 [==============================] - 1s 21ms/step - loss: 14.4347 - val_loss: 14.5909
Epoch 4/100
40/40 [==============================] - 1s 19ms/step - loss: 14.3694 - val_loss: 14.5452
Epoch 5/100
40/40 [==============================] - 1s 20ms/step - loss: 14.3373 - val_loss: 14.5309
Epoch 6/100
40/40 [==============================] - 1s 19ms/step - loss: 14.2630 - val_loss: 14.3441
Epoch 7/100
40/40 [==============================] - 1s 19ms/step - loss: 14.1798 - val_loss: 14.3392
Epoch 8/100
40/40 [==============================] - 1s 18ms/step - loss: 14.1578 - val_loss: 14.2691
Epoch 9/100
40/40 [==============================] - 1s 21ms/step - loss: 14.1093 - val_loss: 14.2734
Epoch 10/100
40/40 [==============================] - 1s 23ms/step - loss: 14.0591

Evaluation

In [25]:
encoded_regs = encoder.predict(test_vecs)
decoded_regs = decoder.predict(encoded_regs)
print(test_strs[:10])
print([v.recover(x) for x in decoded_regs[:10]])
acc = np.sum([v.recover(x) == y for x, y in zip(decoded_regs, test_strs)]) / len(test_strs)
print('accuracy = ' + str(acc))

['AN45FVI', 'LE23DCR', 'HA99JWO', 'EQ58XKI', 'RP49FNK', 'DJ61XIM', 'WO62NON', 'EA42JCE', 'DV47KTZ', 'HM70DFU']
['AN45FVI', 'LE23DCR', 'HA99JWO', 'EM58XKI', 'RP49FNK', 'DJ61XIM', 'WO62NON', 'EA42JCE', 'DV47KTZ', 'HM70DFU']
accuracy = 0.993
